<a href="https://colab.research.google.com/github/afham123/Natural-language-Processing/blob/main/Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import spacy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
nlp=spacy.load('en',disable=['parser','tagger','ner'])

In [72]:
from google.colab import files
uploaded = files.upload()

Saving moby_dick_four_chapters.txt to moby_dick_four_chapters (2).txt


In [73]:
def read_file(file_path):
  with open(file_path) as f:
    str_txt=f.read()
  return str_txt

In [74]:
def seperate_punc(doc):
  return [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ' ]

In [75]:
d=read_file('moby_dick_four_chapters.txt')

In [76]:
tokens = seperate_punc(d)

In [77]:
len(tokens)

11338

In [88]:
train_len = 25+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [89]:
print(text_sequences[0])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']


In [90]:
from sklearn.model_selection import train_test_split

In [91]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [105]:
vocabilary_size=len(tokenizer.word_counts)

In [92]:
print(sequences[0])

[956, 14, 263, 51, 261, 408, 87, 219, 129, 111, 954, 260, 50, 43, 38, 315, 7, 23, 546, 3, 150, 259, 6, 2712, 14, 24]


In [93]:
type(sequences)

list

In [94]:
sequences = np.array(sequences)

In [96]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [97]:
type(sequences)

numpy.ndarray

In [98]:
X=sequences[:,:-1]
y=sequences[:,-1]

In [ ]:
from keras.utils import to_categorical
y=to_categorical(y,vocabilary_size+1)

In [117]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [107]:
seq_len=X_train.shape[1]

In [102]:
print(sequences.shape)
print(X_train.shape)
print(y_train.shape)

(11312, 26)
(7918, 25)
(7918,)


In [108]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [113]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [115]:
model=create_model(vocabilary_size+1,seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_3 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_3 (Dense)              (None, 2718)              410418    
Total params: 787,218
Trainable params: 787,218
Non-trainable params: 0
_________________________________________________________________


In [116]:
from pickle import dump,load

In [ ]:
model.fit(X,y,batch_size=128,epochs=150,verbose=1)

In [ ]:
model.save('my_moby_dick_model.h5')

In [ ]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_ind] 
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    return ' '.join(output_text)